In [7]:
import os
os.environ["GGGI_db_username"] = 'postgres'
os.environ["GGGI_db_password"] = 'lHeJnnRINyWCzfkDOzKl'
os.environ['GGGI_db_endpoint'] ='database-gggi-1.cg4tog4qy0py.ap-northeast-2.rds.amazonaws.com'
os.environ['GGGI_db_port'] = '5432'

In [8]:
import pandas as pd
from sqlalchemy import create_engine

# TO BE HIDDEN in environnement var
username = os.environ.get("GGGI_db_username")
password = os.environ.get("GGGI_db_password")
endpoint = os.environ.get("GGGI_db_endpoint")
port = int(os.environ.get("GGGI_db_port"))
# username = 'postgres'
# password = 'lHeJnnRINyWCzfkDOzKl'
# endpoint = 'database-gggi-1.cg4tog4qy0py.ap-northeast-2.rds.amazonaws.com'
# port = 5432


db_connection_url = f"postgresql://{username}:{password}@{endpoint}:{port}"
engine = create_engine(db_connection_url)

In [9]:
def select_table(tablename, engine=engine):
    sql = f'SELECT * FROM {tablename}'
    with engine.connect() as conn:
        df = pd.read_sql(sql, conn)
    return df


def select_dataset(dataset, engine=engine):
    '''Merge is done localy because otherwise data is too big to be pulled (Unless )
    '''
    df = select_table(dataset)
    meta_df = select_table(f'meta{dataset}')
    return pd.merge(df, meta_df, on='Variable')


def select_all_metadata(engine=engine):
    '''Merge is done localy because otherwise data is too big to be pulled (Unless )
    '''

    metatables = [table for table in engine.table_names() if 'meta' in table]
    dfs = [select_table(table).assign(table=table.replace('meta', '')) for table in metatables]
    
    df = pd.concat(dfs)
    return df

In [10]:
metadf = select_all_metadata().dropna(subset=['Variable']).dropna(axis=1)

In [11]:
metadf.table.unique()

array(['worldbank', 'livestock', 'eia', 'foodbalancesheet_new', 'landuse',
       'MaterialFlow', 'efficientwater', 'crops', 'aquastat',
       'variabledata', 'population', 'cropcalendar', 'evapotranspiration',
       'grossaddedvalue', 'sarahcrops', 'EIA', 'emissions',
       'foodbalancesheet', 'materialflow', 'imfindexcommodity',
       'imfpricecommodity', 'pennworldtable', 'oecdwage'], dtype=object)

In [12]:
metadf

,Description,Variable,Unit,table
0,"Agriculture, forestry, and fishing, value adde...",AGVA,constant 2010 US$,worldbank
1,"Industry (including construction), value added...",IGVA,constant 2010 US$,worldbank
2,"Services, value added (constant 2010 US$)",SGVA,constant 2010 US$,worldbank
3,GDP per capita (constant 2010 US$),GDPC,constant 2010 US$,worldbank
4,"Population, total",Pop,capita,worldbank
...,...,...,...,...
0,"Commodity Export Price Index, Individual Commo...",EPI,1,imfindexcommodity
0,Primary Commodity Price System (PCPS),APCi,$/tonnes,imfpricecommodity
0,Capital stock at current PPPs,PCS,1e6 2017US$,pennworldtable
1,Average depreciation rate of the capital stock,RC,1,pennworldtable
